# Numerical Solution of the Scalar Wave Equation, in Cartesian Coordinates

### NRPy+ Source Code for this module: [ScalarWave/ScalarWave_RHSs.py](../edit/ScalarWave/ScalarWave_RHSs.py); [ScalarWave/InitialData_PlaneWave.py](../edit/ScalarWave/InitialData_PlaneWave.py)

As outlined in the [previous NRPy+ tutorial module](Tutorial-ScalarWave.ipynb), we first use NRPy+ to generate initial data for the scalar wave equation, and then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. <font color='green'>Set gridfunction values to initial data.</font>
1. Evolve the system forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. <font color='green'>Evaluate scalar wave RHS expressions.</font>
    1. Apply boundary conditions.
    1. Print status of the simulation.

### Generating the NRPy+ Components to the Scalar Wave Equation in Three Spatial Dimensions, Tenth-Order Finite Differencing, with Monochromatic Plane Wave Initial Data

Let's pick up where we left off in the [previous module](Tutorial-ScalarWave.ipynb), interfacing with the ScalarWave/ module to generate 
* monochromatic (single-wavelength) plane wave scalar wave initial data, and
* the scalar wave equation RHSs at **10th** finite difference order in **3 spatial dimensions**,

all with SIMD enabled:

In [1]:
# Step P1: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

# Step 1: Import the ScalarWave.InitialData module. 
#         This command only declares ScalarWave initial data 
#         parameters and the InitialData_PlaneWave() function.
import ScalarWave.InitialData_PlaneWave as swid

# Step 2: Import ScalarWave_RHSs module. 
#         This command only declares ScalarWave RHS parameters
#         and the ScalarWave_RHSs function (called later)
import ScalarWave.ScalarWave_RHSs as swrhs

# Step 3: Enable SIMD
par.set_parval_from_str("outputC::SIMD_enable",True)

# Step 4: Set the spatial dimension parameter 
#         to *FOUR* this time, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",3)
DIM = par.parval_from_str("grid::DIM")

# Step 5: Set the finite differencing order to 10.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",10)

# Step 6: Call the InitialData_PlaneWave() function to set up
#         monochromatic (single frequency/wavelength) scalar
#         wave initial data.
swid.InitialData_PlaneWave()

# Step 7: Generate SymPy symbolic expressions for
#         uu_rhs and vv_rhs; the ScalarWave RHSs.
#         This function also declares the uu and vv
#         gridfunctions, which need to be declared
#         to output even the initial data to C file.
swrhs.ScalarWave_RHSs()

# Step 8: Generate C code for the initial data,
#         output to a file named "ScalarWave_PlaneWave_InitialData.h".
fin.FD_outputC("ScalarWave_PlaneWave_InitialData.h",
               [lhrh(lhs=gri.gfaccess("in_gfs","uu"),rhs=swid.uu_ID),
                lhrh(lhs=gri.gfaccess("in_gfs","vv"),rhs=swid.vv_ID)])

# Step 9: Generate C code for scalarwave RHSs,
#         output to a file named "ScalarWave_RHSs.h".
fin.FD_outputC("ScalarWave_RHSs.h",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=swrhs.uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=swrhs.vv_rhs)])

Wrote to file "ScalarWave_PlaneWave_InitialData.h"
Wrote to file "ScalarWave_RHSs.h"


In [1]:
%%writefile "ScalarWave/C_header_files-the_usual_suspects.h"
#include "stdio.h"
#include "stdlib.h"
#include "stdbool.h"
#include "math.h"

Writing "C_header_files-the_usual_suspects.h"


In [2]:
%%writefile "ScalarWave/ScalarWave.c"

#include "C_header_files-the_usual_suspects.h"


int main(int argc,char *argv[]) {
#include "parameter_file_funcs.h"
#include "ID_driver_funcs.h"
#include "Find_Time_Step.h"
#include "timestepping_funcs.h"

    // Read input parameters
    // Set up initial data
    // Find the time step
    // Step forward in time
    return 0;
}

Writing "ScalarWave/ScalarWave.c"


FileNotFoundError: [Errno 2] No such file or directory: '"ScalarWave/ScalarWave.c"'